## ACS Housing Units by Building Size for Urban Villages, City of Phoenix

Keep all building size categories as presented by ACS, add a couple of multifamily summary columns (5-20 units, 20+ units) in case needed for dealing with MOEs and statistical reliability

- https://www.census.gov/data/developers/data-sets/acs-5year.html
- https://api.census.gov/data/2013/acs/acs5/variables.html
- https://api.census.gov/data/2021/acs/acs5/variables.html

In [1]:
import pandas as pd
import math
import numpy as np

In [2]:
import getters as get
import utilcalcs as calc
import geo_agg
from acs_bsize_vars import *

In [3]:
#Search parameters - NOTE DIFFERENT VARIABLE # ASSIGNMENTS IN 2010 VS 2020
y1 = '2021'
y0_5 = '2013'
y0_1 = '2010'

cols_ten = 'group(B25032)'
cols = 'group(B25024)'

source_1 = 'acs/acs1'
#source_5 = 'acs/acs5'

In [4]:
#comp cities and big geo rename dictionary
rename_geos = {'0455000':'Phoenix', '04013':'Maricopa', '0100000US':'US', '0400000US04':'AZ',\
               '0473000':'Tempe','0465000':'Scottsdale', '0427820':'Glendale',\
               '4865000':'San Antonio', '4835000':'Houston', '1235000':'Jacksonville'}

#### Calculate building size columns function

In [5]:
# function to sum select building columns & recalc MOES
#added summary/duplicative counts in creating agg columns to deal with statistical reliability
def group_bsize(df,year):
    df['u_1_E'] = df.loc[:,U1E].sum(axis=1)
    df['u_1_M'] = df.apply(lambda x: (calc.get_moe(x[U1M])),axis=1)
    df['u_24_E'] = df.loc[:,U24E].sum(axis=1)
    df['u_24_M'] = df.apply(lambda x: (calc.get_moe(x[U24M])),axis=1)
    df['u_520_E'] = df.loc[:,U520E].sum(axis=1)
    df['u_520_M'] = df.apply(lambda x: (calc.get_moe(x[U520M])),axis=1)
    df['u_o20_E'] = df.loc[:,Uo20E].sum(axis=1)
    df['u_o20_M'] = df.apply(lambda x: (calc.get_moe(x[Uo20M])),axis=1)
    df['u_oth_E'] = df.loc[:,UOthE].sum(axis=1)
    df['u_oth_M'] = df.apply(lambda x: (calc.get_moe(x[UOthM])),axis=1)
    df = df.rename(hou_rename,axis=1).drop(columns=hou_vars)
    return df

In [6]:
# make city and larger geos
def make_table(source,year,cols,geotype='big'):
    if geotype=='big':
        ph = get.get_phx(source,year,cols)
        mar = get.get_maricopa(source,year,cols)
        us = get.get_us(source,year,cols)
        az = get.get_az(source,year,cols)
        cit = get.get_comp_cities(source,year,cols)
        df = pd.concat([ph,mar,us,az,cit])
        df = df.drop(['NAME','us','state'],axis=1)
        aggtype='GEO_ID'
    elif geotype=='uv':
        df = get.get_bgp(source,year,cols)
        aggtype='name'
    elif geotype=='az_places':
        df = get.get_az_plc(source,year,cols)
        aggtype='GEO_ID'
    else:
        pass
    df = get.clean_table(df)
    df = group_bsize(df,year)
    if geotype=='uv': df = geo_agg.make_uv(df,int(year))
    df = geo_agg.sumgeo_cv(df,aggtype)
    
    for col in df.columns[1:]:
        df.rename(columns={col:f'{col[:-1]}{year[-2:]}{col[-1:]}'},inplace=True)
    return df

## 1-year table for for city & larger geos

In [8]:
geo_y1_1y = make_table(source_1,y1,cols)
#geo_y0_1y = make_table(source_1,y0_1,cols)

In [9]:
big1 = pd.merge(geo_y0_1y,geo_y1_1y,how='left',on='GEO_ID')
big1 = calc.calc_change(big1,y0_1,y1,change_vars)

In [10]:
big1['GEO_ID'] = big1.GEO_ID.map(rename_geos)

## 5-year for AZ places and urban villages

#### urban villages - 5yr

In [11]:
uv_y1_5y = make_table(source_5,y1,cols,'uv')
uv_y0_5y = make_table(source_5,y0_5,cols,'uv')

In [12]:
uv = pd.merge(uv_y0_5y,uv_y1_5y,how='left',on='name')
uv = calc.calc_change(uv,y0_5,y1,change_vars)
uv.rename({'name':'GEO_ID'},axis=1,inplace=True)

#### other places AZ - 5YR

In [13]:
pl_y1_5y = make_table(source_5,y1,cols,'az_places')
pl_y0_5y = make_table(source_5,y0_5,cols,'az_places')

In [14]:
pl = pd.merge(pl_y0_5y,pl_y1_5y,how='left',on='GEO_ID')
pl = calc.calc_change(pl,y0_5,y1,change_vars)

In [16]:
geo_y1_5y = make_table(source_5,y1,cols)
geo_y0_5y = make_table(source_5,y0_5,cols)
big5 = pd.merge(geo_y0_5y,geo_y1_5y,how='left',on='GEO_ID')
big5 = calc.calc_change(big5,y0_5,y1,change_vars)

In [17]:
y5_table = pd.concat([uv,pl,big5])

In [19]:
with pd.ExcelWriter(f'output/acs_blgsize_allgeos.xlsx') as writer:
    big1.to_excel(writer, sheet_name="1yr_big_geos", index=False)
    y5_table.to_excel(writer, sheet_name="5yr_all_geos", index=False)